In [ ]:
def get_test_config_details(lst, dataset_condition):
    ckpt_name_list = lst[1].split('/')
    #print(ckpt_name_list)
    train, mic_idx = ckpt_name_list[-3], ckpt_name_list[-1][-2]

    exp_name_list = lst[-1].split('_')
    #idx = 4 if exp_name_list[-1][0]=='-' else 3
    if dataset_condition=="noisy_reverb":
        snr, t60, test = exp_name_list[-1][:-3], exp_name_list[-3], exp_name_list[-5]
    elif dataset_condition=="reverb":
        snr, t60, test = None, exp_name_list[-1][:-1], exp_name_list[-3]
    elif dataset_condition=="noisy":
        snr, t60, test = exp_name_list[-1][:-3], None, exp_name_list[-3]
    else:
        pass

    return mic_idx, train, test, t60, snr

In [ ]:
# getting metrics for noisy_reverb
import os
from collections import namedtuple

metrics = namedtuple('metrics', ["mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb"])
doa_metrics = namedtuple('doa_metrics', ["mix_frm_Acc", "est_frm_Acc", "mix_blk_Acc", "est_blk_Acc"])

dataset_condition = "noisy"
comparison_domain = "doa"

scenarios = ["stationary", "moving"]
#t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows 
#snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col

files_list = os.listdir(f'../Logs/{dataset_condition}/{comparison_domain}')
exp_dict = {}
doa_exp_dict = {}
for file in files_list:
    if os.path.isfile(os.path.join(f'../Logs/{dataset_condition}/{comparison_domain}', file)) and os.path.splitext(file)[1]==".out":
        with open(os.path.join(f'../Logs/{dataset_condition}/{comparison_domain}', file), 'r') as f:
            for line in f.readlines():
                if "ckpt_dir" in line.strip():
                    mic_idx, train, test, t60, snr = get_test_config_details(line.split(), dataset_condition)
                elif "PESQ_NB_epoch" in line.strip():
                    if "PESQ_NB_epoch"==line.strip().split()[1]:
                        pesq_nb_enh = line.strip().split()[3]
                        #print(pesq_nb_enh)
                    elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                        pesq_nb_mix = line.strip().split()[3]
                        #print(pesq_nb_mix)
                    else:
                        continue
                    #print(line)
                elif "STOI_epoch" in line.strip():
                    if "STOI_epoch"==line.strip().split()[1]:
                        stoi_enh = line.strip().split()[3]
                        #print(stoi_enh)
                    elif "MIX_STOI_epoch"==line.strip().split()[1]:
                        stoi_mix = line.strip().split()[3]
                        #print(stoi_mix)
                    else:
                        continue
                elif "blk_Acc_epoch" in line.strip():
                    if "est_blk_Acc_epoch"==line.strip().split()[1]:
                        est_blk_Acc = line.strip().split()[3]
                    elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                        mix_blk_Acc = line.strip().split()[3]
                    else:
                        continue
                elif "frm_Acc_epoch" in line.strip():
                    if "est_frm_Acc_epoch"==line.strip().split()[1]:
                        est_frm_Acc = line.strip().split()[3]
                    elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                        mix_frm_Acc = line.strip().split()[3]
                    else:
                        continue
                
                #print(line)

    
        
        if comparison_domain =="doa":
            exp_metrics = doa_metrics(mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc)
        else:
            exp_metrics = metrics(stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh)

        #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}

        if dataset_condition =="reverb":
            exp_app_str = f't60_{t60}'
        elif dataset_condition =="noisy":
            exp_app_str = f'snr_{snr}'
        elif dataset_condition =="noisy_reverb":
            exp_app_str = f't60_{t60}_snr_{snr}'
        else:
            exp_app_str = ''

        key_str = f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}' if comparison_domain!="doa" else f'train_{train}_test_{test}_{exp_app_str}'
        exp_dict[key_str] = exp_metrics
        #doa_exp_dict[f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}'] = doa_metrics

        #break




In [ ]:

#import torch
#torch.save(exp_dict, f"{dataset_condition}_doa_results.pt")


In [ ]:
mic_idx=0
#dataset_condition = "reverb"
train="stationary"
test = "stationary"
t60=0.8
snr = -5.0
if dataset_condition =="reverb":
    exp_app_str = f't60_{t60}'
elif dataset_condition =="noisy":
    exp_app_str = f'snr_{snr}'
elif dataset_condition =="noisy_reverb":
    exp_app_str = f't60_{t60}_snr_{snr}'
else:
    exp_app_str = ''

key_str = f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}' if comparison_domain!="doa" else f'train_{train}_test_{test}_{exp_app_str}'
exp_dict[key_str]

In [ ]:
exp_dict

In [ ]:
len(exp_dict)

In [ ]:
import torch
dataset_condition = "noisy"
comparison_domain="se"
exp_dict = torch.load(f'../Logs/{dataset_condition}/{dataset_condition}_{comparison_domain}_results.pt')

In [ ]:
#MIMO

# getting metrics for noisy_reverb
import os
from collections import namedtuple

metrics = namedtuple('metrics', ["mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb", "mix_frm_Acc", "est_frm_Acc", "mix_blk_Acc", "est_blk_Acc"])


dataset_condition = "noisy_reverb"
comparison_domain = "doa"

scenarios = ["stationary", "moving"]
#t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows 
#snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col

files_list = os.listdir(f'../Logs/{dataset_condition}/diffuse_noise/miso_ri_mag')
exp_dict = {}
doa_exp_dict = {}
for file in files_list:
    if os.path.isfile(os.path.join(f'../Logs/{dataset_condition}/diffuse_noise/miso_ri_mag', file)) and os.path.splitext(file)[1]==".out":
        try:
            with open(os.path.join(f'../Logs/{dataset_condition}/diffuse_noise/miso_ri_mag', file), 'r') as f:
                for line in f.readlines():
                    if "ckpt_dir" in line.strip():
                        mic_idx, train, test, t60, snr = get_test_config_details(line.split(), dataset_condition)
                    elif "PESQ_NB_epoch" in line.strip():
                        if "PESQ_NB_epoch"==line.strip().split()[1]:
                            pesq_nb_enh = line.strip().split()[3]
                            #print(pesq_nb_enh)
                        elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                            pesq_nb_mix = line.strip().split()[3]
                            #print(pesq_nb_mix)
                        else:
                            continue
                        #print(line)
                    elif "STOI_epoch" in line.strip():
                        if "STOI_epoch"==line.strip().split()[1]:
                            stoi_enh = line.strip().split()[3]
                            #print(stoi_enh)
                        elif "MIX_STOI_epoch"==line.strip().split()[1]:
                            stoi_mix = line.strip().split()[3]
                            #print(stoi_mix)
                        else:
                            continue
                    elif "blk_Acc_epoch" in line.strip():
                        if "est_blk_Acc_epoch"==line.strip().split()[1]:
                            est_blk_Acc = line.strip().split()[3]
                        elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                            mix_blk_Acc = line.strip().split()[3]
                        else:
                            continue
                    elif "frm_Acc_epoch" in line.strip():
                        if "est_frm_Acc_epoch"==line.strip().split()[1]:
                            est_frm_Acc = line.strip().split()[3]
                        elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                            mix_frm_Acc = line.strip().split()[3]
                        else:
                            continue
                    
                    #print(line)
        except UnicodeDecodeError:
            print(file)
            #t60 = 1.0
            #snr = 10.0
            #train = "stationary"
            #test = "moving"
            #stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc = 0,0,0,0,0,0,0,0


    
        

        exp_metrics = metrics(stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc)

        #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}

        if dataset_condition =="reverb":
            exp_app_str = f't60_{t60}'
        elif dataset_condition =="noisy":
            exp_app_str = f'snr_{snr}'
        elif dataset_condition =="noisy_reverb":
            exp_app_str = f't60_{t60}_snr_{snr}'
        else:
            exp_app_str = ''

        key_str = f'train_{train}_test_{test}_{exp_app_str}' 
        exp_dict[key_str] = exp_metrics
        #doa_exp_dict[f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}'] = doa_metrics

        #break




In [ ]:
exp_dict

In [ ]:
import torch
torch.save(exp_dict, f"{dataset_condition}_diffuse_noise_miso_ri_mag_results.pt")

In [ ]:
len(exp_dict)

In [ ]:
exp_dict['train_moving_test_stationary_t60_1.0_snr_-5.0']

In [ ]:
def get_test_config_details_v2(lst, dataset_condition):
    ckpt_name_list = lst[1].split('/')
    #print(ckpt_name_list)
    loss_function, train, mic_idx = ckpt_name_list[-4], ckpt_name_list[-3], ckpt_name_list[-1][-2] 
    if "stationary"!=train:
        loss_function, train, mic_idx = ckpt_name_list[-5], ckpt_name_list[-4], ckpt_name_list[-1][-2] #

    exp_name_list = lst[-1].split('_')
    #idx = 4 if exp_name_list[-1][0]=='-' else 3
    if dataset_condition=="noisy_reverb":
        snr, t60, test = exp_name_list[-1][:-3], exp_name_list[-3], exp_name_list[-5]
    elif dataset_condition=="reverb":
        snr, t60, test = None, exp_name_list[-1][:-1], exp_name_list[-3]
    elif dataset_condition=="noisy":
        snr, t60, test = exp_name_list[-1][:-3], None, exp_name_list[-3]
    else:
        pass

    return mic_idx, train, test, t60, snr, loss_function

In [ ]:
def get_metrics_lst(base_dir, array_info):

    files_list = os.listdir(base_dir)
    exp_dict = {}
    doa_exp_dict = {}
    exp_metrics_lst = []
    percent_format= lambda x: round(float(x)*100, 2) 
    frac_format= lambda x:  round(float(x), 2)

    for file in files_list:
        if os.path.isfile(os.path.join(base_dir, file)) and os.path.splitext(file)[1]==".out":
            try:
                with open(os.path.join(base_dir, file), 'r') as f:
                    for line in f.readlines():
                        if "ckpt_dir" in line.strip():
                            mic_idx, train, test, t60, snr, loss_function = get_test_config_details_v2(line.split(), dataset_condition)
                            
                        elif "PESQ_NB_epoch" in line.strip():
                            if "PESQ_NB_epoch"==line.strip().split()[1]:
                                pesq_nb_enh = line.strip().split()[3]

                                #print(pesq_nb_enh)
                            elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                                pesq_nb_mix = line.strip().split()[3]
                                #print(pesq_nb_mix)
                            else:
                                continue
                            #print(line)
                        elif "STOI_epoch" in line.strip():
                            if "STOI_epoch"==line.strip().split()[1]:
                                stoi_enh = line.strip().split()[3]
                
                            elif "MIX_STOI_epoch"==line.strip().split()[1]:
                                stoi_mix = line.strip().split()[3]
                                #print(stoi_mix)
                            else:
                                continue
                        elif "blk_Acc_epoch" in line.strip():
                            if "est_blk_Acc_epoch"==line.strip().split()[1]:
                                est_blk_Acc = line.strip().split()[3]
                            elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                                mix_blk_Acc = line.strip().split()[3]
                            else:
                                continue
                        elif "frm_Acc_epoch" in line.strip():
                            if "est_frm_Acc_epoch"==line.strip().split()[1]:
                                est_frm_Acc = line.strip().split()[3]
                            elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                                mix_frm_Acc = line.strip().split()[3]
                            elif "est_2mic_frm_Acc_epoch"==line.strip().split()[1]:
                                est_2mic_frm_Acc = line.strip().split()[3]
                            elif "mix_2mic_frm_Acc_epoch"==line.strip().split()[1]:
                                mix_2mic_frm_Acc = line.strip().split()[3]
                            else:
                                continue

                        
                        
                        #print(line)
            except UnicodeDecodeError:
                print(file)
                #t60 = 1.0
                #snr = 10.0
                #train = "stationary"
                #test = "moving"
                #stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc = 0,0,0,0,0,0,0,0

            exp_metrics = metrics(array_info, loss_function, dataset_condition, noise_simulation, t60, snr, 
                                  percent_format(stoi_mix), percent_format(stoi_enh), frac_format(pesq_nb_mix), frac_format(pesq_nb_enh), 
                                  percent_format(mix_frm_Acc), percent_format(est_frm_Acc), percent_format(mix_blk_Acc), percent_format(est_blk_Acc), 
                                  percent_format(mix_2mic_frm_Acc), percent_format(est_2mic_frm_Acc))
        
            #df = pd.DataFrame(metrics(loss_function, dataset_condition, noise_simulation, t60, snr, stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc, mix_2mic_frm_Acc, est_2mic_frm_Acc))
            #videos_df = pd.concat(videos, ignore_index=True)
            #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}

            if dataset_condition =="reverb":
                exp_app_str = f't60_{t60}'
            elif dataset_condition =="noisy":
                exp_app_str = f'snr_{snr}',
            elif dataset_condition =="noisy_reverb":
                exp_app_str = f't60_{t60}_snr_{snr}'
            else:
                exp_app_str = ''

            key_str = f'{loss_function}_train_{train}_test_{test}_{exp_app_str}' 
            #print(key_str)
            exp_dict[key_str] = exp_metrics
            #doa_exp_dict[f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}'] = doa_metrics

            #break

            #list (fro dataframe)
            exp_metrics_lst.append(exp_metrics)

    return exp_metrics_lst


In [20]:
#Loss functions comparison

#MIMO

# getting metrics for noisy_reverb
import os
from collections import namedtuple
import pandas as pd


metrics = namedtuple('metrics', ["Array_type_num_mic_intermic_dist", "loss_function", "dataset_condition", "noise_simulation", "T60", "SNR", "mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb", "mix_frm_Acc", "est_frm_Acc",  "mix_2mic_frm_Acc", "est_2mic_frm_Acc"]) #"mix_blk_Acc", "est_blk_Acc",

dataset_condition = "reverb" #"noisy" #
noise_simulations = ["point_source"] # ,"diffuse"
comparison_domain = "doa"
num_mic = 4

scenarios = ["stationary"]#, "moving"]
#t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows 
#snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col
df_list = []
for idx, noise_simulation in enumerate(noise_simulations):
    base_dir = f'../Logs/8cm_array_4mic/{dataset_condition}/simu_rirs' #{noise_simulation}/simu_rirs_mask_loss_doa_mask'

    df_list.append(pd.DataFrame(get_metrics_lst(base_dir, array_info = "Linear_4mic_8cm")))

df = pd.concat(df_list)


In [21]:
df
#df.to_csv("../Logs/8cm_array_2mic/noisy_reverb_ps_mask_loss_doa_mask.csv")

,Array_type_num_mic_intermic_dist,loss_function,dataset_condition,noise_simulation,T60,SNR,mix_stoi,enh_stoi,mix_pesq_nb,enh_pesq_nb,mix_frm_Acc,est_frm_Acc,mix_2mic_frm_Acc,est_2mic_frm_Acc
0,Linear_4mic_8cm,MIMO_RI,reverb,point_source,0.,None,80.49,93.89,2.24,2.81,62.46,71.60,41.92,63.97
1,Linear_4mic_8cm,MIMO_RI_MAG_PD,reverb,point_source,0.,None,80.49,93.96,2.24,2.94,62.46,82.23,41.92,76.50
2,Linear_4mic_8cm,MIMO_RI,reverb,point_source,0.,None,92.24,96.65,3.03,3.26,88.15,80.84,73.71,72.30
3,Linear_4mic_8cm,MIMO_RI_MAG_PD,reverb,point_source,0.,None,92.24,95.34,3.03,3.11,88.15,84.35,73.71,80.16
4,Linear_4mic_8cm,MIMO_RI,reverb,point_source,0.,None,100.00,98.56,4.50,3.78,100.00,85.24,100.00,73.29
5,Linear_4mic_8cm,MIMO_RI_MAG_PD,reverb,point_source,0.,None,100.00,94.07,4.50,3.18,100.00,86.85,100.00,82.06
6,Linear_4mic_8cm,MIMO_RI,reverb,point_source,0.,None,85.79,95.17,2.51,2.97,73.24,75.62,53.00,67.74
7,Linear_4mic_8cm,MIMO_RI_MAG_PD,reverb,point_source,0.,None,85.79,94.82,2.51,3.04,73.24,83.14,53.00,78.12
8,Linear_4mic_8cm,MIMO_RI,reverb,point_source,0.,None,76.02,92.76,2.07,2.69,54.86,68.58,35.33,60.92
9,Linear_4mic_8cm,MIMO_RI_MAG_PD,reverb,point_source,0.,None,76.02,93.03,2.07,2.84,54.86,81.54,35.33,75.21


In [19]:
def get_test_config_details_v2_loss_functions(lst, dataset_condition):
    ckpt_name_list = lst[1].split('/')
    #print(ckpt_name_list)
    loss_function, train, mic_idx = ckpt_name_list[-4], ckpt_name_list[-3], ckpt_name_list[-1][-2] 
    if "stationary"!=train:
        loss_function, train, mic_idx = ckpt_name_list[-5], ckpt_name_list[-4], ckpt_name_list[-1][-2] #

    exp_name_list = lst[-1].split('_')
    #idx = 4 if exp_name_list[-1][0]=='-' else 3
    if dataset_condition=="noisy_reverb":
        snr, t60, test = exp_name_list[-1][:-3], exp_name_list[-3], exp_name_list[-5]
    elif dataset_condition=="reverb":
        snr, t60, test = None, exp_name_list[-5][:], exp_name_list[-3]
    elif dataset_condition=="noisy":
        snr, t60, test = exp_name_list[-4][:-2], None, exp_name_list[-6]
    else:
        pass

    return mic_idx, train, test, t60, snr, loss_function

In [16]:
#all in one loss functions comparison
loss_list = ["MIMO_RI", "MIMO_RI_MAG", "MIMO_RI_MAG_PD", "MIMO_RI_PD"]#, "MIMO_RI_PD_REF"]

def get_metrics_lst(base_dir, array_info):

    files_list = os.listdir(base_dir) 
    exp_dict = {}
    doa_exp_dict = {}
    exp_metrics_lst = []
    percent_format= lambda x: round(float(x)*100, 2) 
    frac_format= lambda x:  round(float(x), 2)

    for file in files_list:
        if 'functions_comparison' in file and os.path.isfile(os.path.join(base_dir, file)) and os.path.splitext(file)[1]==".out": #loss_
            try:
                with open(os.path.join(base_dir, file), 'r') as f:
                    for line in f.readlines():
                        if "ckpt_dir" in line.strip():
                            mic_idx, train, test, t60, snr, _ = get_test_config_details_v2_loss_functions(line.split(), dataset_condition)
                            
                        elif "PESQ_NB_epoch" in line.strip():
                            if "MIMO_RI_PESQ_NB_epoch"==line.strip().split()[1]:
                                ri_pesq_nb_enh = line.strip().split()[3]
                            elif "MIMO_RI_MAG_PESQ_NB_epoch"==line.strip().split()[1]:
                                ri_mag_pesq_nb_enh = line.strip().split()[3]
                            elif "MIMO_RI_MAG_PD_PESQ_NB_epoch"==line.strip().split()[1]:
                                ri_mag_pd_pesq_nb_enh = line.strip().split()[3]
                            elif "MIMO_RI_PD_PESQ_NB_epoch"==line.strip().split()[1]:
                                ri_pd_pesq_nb_enh = line.strip().split()[3]
                                #print(pesq_nb_enh)
                            elif "MIMO_RI_PD_REF_PESQ_NB_epoch"==line.strip().split()[1]:
                                ri_pd_ref_pesq_nb_enh = line.strip().split()[3]
                            elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                                pesq_nb_mix = line.strip().split()[3]
                                #print(pesq_nb_mix)
                            else:
                                continue
                            #print(line)
                        elif "STOI_epoch" in line.strip():
                            if "MIMO_RI_STOI_epoch"==line.strip().split()[1]:
                                ri_stoi_enh = line.strip().split()[3]
                            elif "MIMO_RI_MAG_STOI_epoch"==line.strip().split()[1]:
                                ri_mag_stoi_enh = line.strip().split()[3]
                            elif "MIMO_RI_MAG_PD_STOI_epoch"==line.strip().split()[1]:
                                ri_mag_pd_stoi_enh = line.strip().split()[3]
                            elif "MIMO_RI_PD_STOI_epoch"==line.strip().split()[1]:
                                ri_pd_stoi_enh = line.strip().split()[3] 
                            elif "MIMO_RI_PD_REF_STOI_epoch"==line.strip().split()[1]:
                                ri_pd_ref_stoi_enh = line.strip().split()[3] 
                            elif "MIX_STOI_epoch"==line.strip().split()[1]:
                                stoi_mix = line.strip().split()[3]
                                #print(stoi_mix)
                            else:
                                continue
                        elif "blk_Acc_epoch" in line.strip():
                            if "est_blk_Acc_epoch"==line.strip().split()[1]:
                                est_blk_Acc = line.strip().split()[3]
                            elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                                mix_blk_Acc = line.strip().split()[3]
                            else:
                                continue
                        elif "frm_Acc" in line.strip(): #ch
                            if "MIMO_RI_est_frm_Acc_epoch"==line.strip().split()[1]:
                                ri_est_frm_Acc = line.strip().split()[3]
                            elif "MIMO_RI_MAG_est_frm_Acc_epoch"==line.strip().split()[1]:
                                ri_mag_est_frm_Acc = line.strip().split()[3]
                            elif "MIMO_RI_MAG_PD_est_frm_Acc_epoch"==line.strip().split()[1]:
                                ri_mag_pd_est_frm_Acc = line.strip().split()[3]
                            elif "MIMO_RI_PD_est_frm_Acc_epoch"==line.strip().split()[1]:
                                ri_pd_est_frm_Acc = line.strip().split()[3]
                            elif "MIMO_RI_PD_REF_est_frm_Acc_epoch"==line.strip().split()[1]:
                                ri_pd_ref_est_frm_Acc = line.strip().split()[3]
                            elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                                mix_frm_Acc = line.strip().split()[3]
                            elif "MIMO_RI_est_2mic_frm_Acc_epoch"==line.strip().split()[1]:
                                ri_est_2mic_frm_Acc = line.strip().split()[3]
                            elif "MIMO_RI_MAG_est_2mic_frm_Acc_epoch"==line.strip().split()[1]:
                                ri_mag_est_2mic_frm_Acc = line.strip().split()[3]
                            elif "MIMO_RI_MAG_PD_est_2mic_frm_Acc" in line.strip().split()[1]: #ch
                                ri_mag_pd_est_2mic_frm_Acc = line.strip().split()[3]
                            elif "MIMO_RI_PD_est_2mic_frm_Acc" in line.strip().split()[1]:
                                ri_pd_est_2mic_frm_Acc = line.strip().split()[3]
                            elif "MIMO_RI_PD_REF_est_2mic_frm_Acc" in line.strip().split()[1]:
                                ri_pd_ref_est_2mic_frm_Acc = line.strip().split()[3]
                            elif "mix_2mic_frm_Acc_epoch"==line.strip().split()[1]:
                                mix_2mic_frm_Acc = line.strip().split()[3]
                            else:
                                continue

                        
                        
                        #print(line)
            except UnicodeDecodeError:
                print(file)
                #t60 = 1.0
                #snr = 10.0
                #train = "stationary"
                #test = "moving"
                #stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc = 0,0,0,0,0,0,0,0
            
            ri_exp_metrics = metrics(array_info, "MIMO_RI", dataset_condition, noise_simulation, t60, snr, 
                                  percent_format(stoi_mix), percent_format(ri_stoi_enh), frac_format(pesq_nb_mix), frac_format(ri_pesq_nb_enh), 
                                  percent_format(mix_frm_Acc), percent_format(ri_est_frm_Acc), 
                                    percent_format(mix_2mic_frm_Acc), percent_format(ri_est_2mic_frm_Acc))
            """
            ri_mag_exp_metrics = metrics(array_info, "MIMO_RI_MAG", dataset_condition, noise_simulation, t60, snr, 
                                  percent_format(stoi_mix), percent_format(ri_mag_stoi_enh), frac_format(pesq_nb_mix), frac_format(ri_mag_pesq_nb_enh), 
                                  percent_format(mix_frm_Acc), percent_format(ri_mag_est_frm_Acc), 
                                    percent_format(mix_2mic_frm_Acc), percent_format(ri_mag_est_2mic_frm_Acc))
            """
            ri_mag_pd_exp_metrics = metrics(array_info, "MIMO_RI_MAG_PD", dataset_condition, noise_simulation, t60, snr, 
                                  percent_format(stoi_mix), percent_format(ri_mag_pd_stoi_enh), frac_format(pesq_nb_mix), frac_format(ri_mag_pd_pesq_nb_enh), 
                                  percent_format(mix_frm_Acc), percent_format(ri_mag_pd_est_frm_Acc), 
                                    percent_format(mix_2mic_frm_Acc), percent_format(ri_mag_pd_est_2mic_frm_Acc))
            """
            ri_pd_exp_metrics = metrics(array_info, "MIMO_RI_PD", dataset_condition, noise_simulation, t60, snr, 
                                  percent_format(stoi_mix), percent_format(ri_pd_stoi_enh), frac_format(pesq_nb_mix), frac_format(ri_pd_pesq_nb_enh), 
                                  percent_format(mix_frm_Acc), percent_format(ri_pd_est_frm_Acc), 
                                    percent_format(mix_2mic_frm_Acc), percent_format(ri_pd_est_2mic_frm_Acc))
            
            ri_pd_ref_exp_metrics = metrics(array_info, "MIMO_RI_PD_REF", dataset_condition, noise_simulation, t60, snr, 
                                  percent_format(stoi_mix), percent_format(ri_pd_ref_stoi_enh), frac_format(pesq_nb_mix), frac_format(ri_pd_ref_pesq_nb_enh), 
                                  percent_format(mix_frm_Acc), percent_format(ri_pd_ref_est_frm_Acc), 
                                    percent_format(mix_2mic_frm_Acc), percent_format(ri_pd_ref_est_2mic_frm_Acc))
            """
            #df = pd.DataFrame(metrics(loss_function, dataset_condition, noise_simulation, t60, snr, stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc, mix_2mic_frm_Acc, est_2mic_frm_Acc))
            #videos_df = pd.concat(videos, ignore_index=True)
            #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}


            #list (fro dataframe)
            exp_metrics_lst.append(ri_exp_metrics)
            #exp_metrics_lst.append(ri_mag_exp_metrics)
            exp_metrics_lst.append(ri_mag_pd_exp_metrics)
            #exp_metrics_lst.append(ri_pd_exp_metrics)
            #exp_metrics_lst.append(ri_pd_ref_exp_metrics)

    return exp_metrics_lst


In [ ]:
#Loss functions comparison

#MIMO

# getting metrics for noisy_reverb
import os
from collections import namedtuple
import pandas as pd


metrics = namedtuple('metrics', ["loss_function", "dataset_condition", "noise_simulation", "T60", "SNR", "mix_stoi", "enh_stoi", "mix_pesq_nb", "enh_pesq_nb", "mix_frm_Acc", "est_frm_Acc", "mix_blk_Acc", "est_blk_Acc", "mix_2mic_frm_Acc", "est_2mic_frm_Acc"])



dataset_condition = "reverb"#"noisy_reverb"
noise_simulation = "point_source"  #"diffuse" #
comparison_domain = "doa"

scenarios = ["stationary"]#, "moving"]
#t60_list = [0.2, 0.4, 0.6, 0.8, 1.0] # rows 
#snr_list = [-5.0, 0.0, 5.0, 10.0, 20.0] # col

base_dir = f'../Logs/8cm_array_4mic/{dataset_condition}/{noise_simulation}/simu_rirs' #_doa_mask'

files_list = os.listdir(base_dir)
exp_dict = {}
doa_exp_dict = {}
exp_metrics_lst = []

for file in files_list:
    if os.path.isfile(os.path.join(base_dir, file)) and os.path.splitext(file)[1]==".out":
        try:
            with open(os.path.join(base_dir, file), 'r') as f:
                for line in f.readlines():
                    if "ckpt_dir" in line.strip():
                        mic_idx, train, test, t60, snr, loss_function = get_test_config_details_v2(line.split(), dataset_condition)
                        
                    elif "PESQ_NB_epoch" in line.strip():
                        if "PESQ_NB_epoch"==line.strip().split()[1]:
                            pesq_nb_enh = line.strip().split()[3]
                            #print(pesq_nb_enh)
                        elif "MIX_PESQ_NB_epoch"==line.strip().split()[1]:
                            pesq_nb_mix = line.strip().split()[3]
                            #print(pesq_nb_mix)
                        else:
                            continue
                        #print(line)
                    elif "STOI_epoch" in line.strip():
                        if "STOI_epoch"==line.strip().split()[1]:
                            stoi_enh = line.strip().split()[3]
                            
                        elif "MIX_STOI_epoch"==line.strip().split()[1]:
                            stoi_mix = line.strip().split()[3]
                            #print(stoi_mix)
                        else:
                            continue
                    elif "blk_Acc_epoch" in line.strip():
                        if "est_blk_Acc_epoch"==line.strip().split()[1]:
                            est_blk_Acc = line.strip().split()[3]
                        elif "mix_blk_Acc_epoch"==line.strip().split()[1]:
                            mix_blk_Acc = line.strip().split()[3]
                        else:
                            continue
                    elif "frm_Acc_epoch" in line.strip():
                        if "est_frm_Acc_epoch"==line.strip().split()[1]:
                            est_frm_Acc = line.strip().split()[3]
                        elif "mix_frm_Acc_epoch"==line.strip().split()[1]:
                            mix_frm_Acc = line.strip().split()[3]
                        elif "est_2mic_frm_Acc_epoch"==line.strip().split()[1]:
                            est_2mic_frm_Acc = line.strip().split()[3]
                        elif "mix_2mic_frm_Acc_epoch"==line.strip().split()[1]:
                            mix_2mic_frm_Acc = line.strip().split()[3]
                        else:
                            continue

                    
                    
                    #print(line)
        except UnicodeDecodeError:
            print(file)
            #t60 = 1.0
            #snr = 10.0
            #train = "stationary"
            #test = "moving"
            #stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc = 0,0,0,0,0,0,0,0

        exp_metrics = metrics(loss_function, dataset_condition, noise_simulation, t60, snr, stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc, mix_2mic_frm_Acc, est_2mic_frm_Acc)
       
        #df = pd.DataFrame(metrics(loss_function, dataset_condition, noise_simulation, t60, snr, stoi_mix, stoi_enh, pesq_nb_mix, pesq_nb_enh, mix_frm_Acc, est_frm_Acc, mix_blk_Acc, est_blk_Acc, mix_2mic_frm_Acc, est_2mic_frm_Acc))
        #videos_df = pd.concat(videos, ignore_index=True)
        #exp_metrics = { "mix_stoi": stoi_mix, "enh_stoi": stoi_enh, "mix_pesq_nb": pesq_nb_mix, "enh_pesq_nb": pesq_nb_enh}

        if dataset_condition =="reverb":
            exp_app_str = f't60_{t60}'
        elif dataset_condition =="noisy":
            exp_app_str = f'snr_{snr}',
        elif dataset_condition =="noisy_reverb":
            exp_app_str = f't60_{t60}_snr_{snr}'
        else:
            exp_app_str = ''

        key_str = f'{loss_function}_train_{train}_test_{test}_{exp_app_str}' 
        #print(key_str)
        exp_dict[key_str] = exp_metrics
        #doa_exp_dict[f'train_{train}_mic_idx_{mic_idx}_test_{test}_{exp_app_str}'] = doa_metrics

        #break

        #list (fro dataframe)
        exp_metrics_lst.append(exp_metrics)



In [ ]:
pd.DataFrame(exp_metrics_lst)

In [ ]:
len(exp_dict)

In [ ]:
loss_list = ["MIMO_RI", "MIMO_RI_MAG", "MIMO_RI_MAG_PD", "MIMO_RI_PD", "MIMO_RI_PD_REF"]
t60, snr = 1.0, 0.0
for _loss in loss_list:
    key_str = f'{_loss}_train_{train}_test_{test}_t60_{t60}_snr_{snr}' 
    print(exp_dict[key_str])

In [ ]:
exp_dict

In [ ]:
exp_dict

In [ ]:
type(stoi_enh)